In [1]:
# | hide

from IPython.display import Markdown as md
from IPython.display import Code


# Cryptocurrency analysis with FastStream

In this tutorial, we will walk through the process of using the `faststream-gen` Python library to retrieve cryptocurrency prices in real time and calculate their moving average. To accomplish that, we will generate following two <a href="https://faststream.airt.ai" target="_blank">FastStream</a> applications:

1. A microservice retrieving current cryptocurrency prices from an external <a href="https://help.coinbase.com/en/cloud/api/coinbase" target="_blank">web service</a> and publishing retrieved data to a Kafka topic.

2. A microservice consuming such messages, calculating the moving average price for each cryptocurrency and publishing it to another Kafka topic.

**Let's get started!**

## Installation

To complete this tutorial, you will need the following software and Python library:

1. <a href="https://www.python.org/" target="_blank">Python</a> (version 3.8 and upward)

2. a valid <a href="https://platform.openai.com/account/api-keys" target="_blank">OPENAI API key</a>

3. [optional] <a href="https://github.com/" target="_blank">github account</a> and installed <a href="https://git-scm.com/" target="_blank">git command</a>


It is recommended to use a virtual environment for your Python projects. Virtual environments are a common and effective Python development technique that helps to keep dependencies required by different projects separate by creating isolated Python environments for them.

In this tutorial, we will be using Python’s venv module to create a virtual environment.

First, create a root directory for this tutorial. Navigate to the desired location and create a new directory called `faststream_gen_tutorial` and enter it.

```sh
mkdir faststream_gen_tutorial
cd faststream_gen_tutorial
```

### Creating and activating a new Python virtual environment

Create a new virtual environment using <a href="https://docs.python.org/3/library/venv.html" target="_blank"> venv</a>:
```sh
python3 -m venv venv
```

Next, activate your new virtual environment:

```sh
source venv/bin/activate
```

### Installing the packages

Upgrade pip if needed and install `faststream-gen` package:

```sh
pip install --upgrade pip & pip install faststream-gen

```

Check that the installation was succesful by running the following command:
```sh
faststream_gen --help
```
You should see the full list of options of the command in the output.

Now you have successfully set up the environment and installed the `faststream-gen` package. You are ready to start using it!

### Setting up OpenAI API key

`faststream-gen` uses OpenAI API and you need to export your API key in environment variable `OPENAI_API_KEY`. If you use bash or compatible shell, you can do that with the following command:

```sh
export OPENAI_API_KEY="<your_openai_api_key>"
```

If you don't already have `OPENAI_API_KEY`, you can create one <a href="https://platform.openai.com/account/api-keys" target="_blank">here</a>.

## Generate FastStream apps

### Retrieve and publish crypto prices

To simplify the implementation and integration process of the `FastStream` application, we have created a GitHub template repository. To proceed, please open the ([template](https://github.com/airtai/faststream-template)) and click on `Use this template` followed by `Create a new repository`. For the repository name, use `retrieve-publish-cryptocrypto-fetch-tutorial`.
Once the repository creation is finished, copy the URL of your repository. Return to your development environment, execute the `git clone` command and enter the repository.
```sh
git clone https://github.com/your-username/retrieve-publish-cryptocrypto-retrieve-tutorial.git
cd retrieve-publish-crypto
```

Now, we will create an application that retrieves information about cryptocurrencies from various web sources and publishes messages to a Kafka topic. In order to achieve this, we will provide a comprehensive explanation of the necessary implementation steps. This will include details such as the message schema, instructions on obtaining cryptocurrency prices, and guidance on selecting the appropriate topic and partition keys.

Here is the full description of the desired application:

In [18]:
# | hide

with open('../../docs_src/tutorial/retrieve-publish-crypto/description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [19]:
# | echo: false

md(description)


```text
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

The current price of Bitcoin can be retrieved by a simple GET request to:
    - https://api.coinbase.com/v2/prices/BTC-USD/spot
    
The current price of Ethereum can be retrieved by a simple GET request to:
    - https://api.coinbase.com/v2/prices/ETH-USD/spot

The response of this GET request is a JSON and you can get
information about the crypto_currency in:
    response['data']['base']
    
and the information about the price in:
    response['data']['amount']

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.
 
```


Let's generate a new `FastStream` application inside the `retrieve-publish-crypto` directory. First, copy the previous description and paste it into a file called `description.txt` in the current (`retrieve-publish-crypto`) working directory.

Next, run the following command (parameter `-i` specifies the filepath for the app description file):

```sh
faststream_gen -i description.txt
```
```console
✨  Generating a new FastStream application!
 ✔ Application description validated. 
 ✔ FastStream app skeleton code generated. 
 ✔ The app and the tests are generated. 
 ✔ New FastStream project created. 
 ✔ Integration tests were successfully completed. 
 Tokens used: 36938
 Total Cost (USD): $0.11436
✨  All files were successfully generated!
```

!!! note

    By default, faststream_gen utilizes `gpt-3.5-turbo` for creating FastStream applications. If you encounter any generation issues, we recommend that you try again with the `--model gpt-4` option. With gpt-4's enhanced capabilities, you can expect even higher success rates, especially when dealing with more complex tasks. 



Once the generation is complete, you can find the application in the `app` folder and the tests in the `tests` folder.

`app/application.py`:

In [20]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/retrieve-publish-crypto/app/application.py', language='python')
md(f"```python\n{code}\n```")

```python
from pydantic import BaseModel, Field, NonNegativeFloat
from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker
import requests
import asyncio

class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., examples=[10000.0], description="Current price of cryptocurrency in USD")
    crypto_currency: str = Field(..., examples=["BTC"], description="The cryptocurrency")

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

publisher = broker.publisher("new_crypto_price")


@broker.publisher("new_crypto_price")
async def publish_crypto_price(logger: Logger, crypto_currency: str, price: float) -> None:
    new_crypto_price = CryptoPrice(price=price, crypto_currency=crypto_currency)
    await publisher.publish(new_crypto_price, key=crypto_currency.encode("utf-8"))


@app.on_startup
async def app_setup():
    app.set_global("app_is_running", True)


@app.on_shutdown
async def shutdown():
    app.set_global("app_is_running", False)


async def fetch_crypto_price(crypto_currency: str, logger: Logger) -> None:
    while app.get_global("app_is_running"):
        url = f"https://api.coinbase.com/v2/prices/{crypto_currency}-USD/spot"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            price = data["data"]["amount"]
            await publish_crypto_price(logger, crypto_currency, price)
        else:
            logger.warning(f"Failed API request {url}")

        await asyncio.sleep(2)


@app.after_startup
async def start_fetching(logger: Logger):
    crypto_currencies = ["BTC", "ETH"]
    fetch_tasks = [
        asyncio.create_task(fetch_crypto_price(crypto_currency, logger))
        for crypto_currency in crypto_currencies
    ]
    app.set_global("fetch_tasks", fetch_tasks)
```

`tests/test_application.py`

In [21]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/retrieve-publish-crypto/tests/test_application.py', language='python')
md(f"```python\n{code}\n```")

```python
import pytest

from faststream import Context, TestApp
from faststream.kafka import TestKafkaBroker

from app.application import CryptoPrice, app, broker


@broker.subscriber("new_crypto_price")
async def on_new_crypto_price(msg: CryptoPrice, key: bytes = Context("message.raw_message.key")):
    pass


@pytest.mark.asyncio
async def test_message_was_published():
    async with TestKafkaBroker(broker):
        async with TestApp(app):
            await on_new_crypto_price.wait_call(2)
            on_new_crypto_price.mock.assert_called()
```

#### Test

In order to verify functional correctness  of the application, it is recommended to execute the generated unit and integration test by running the  `pytest` command.

```sh
pytest
```

In [14]:
# | echo: false

! cd /work/fastkafka-gen/docs_src/tutorial/retrieve-publish-crypto && pytest

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: /work/fastkafka-gen/docs_src/tutorial/retrieve-publish-crypto
configfile: pyproject.toml
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item                                                               

tests/test_application.py .                                              [100%]

============================== 1 passed in 2.80s ===============================


#### Start localhost Kafka broker

To run the `FastStream` application locally, ensure that you have a running Kafka broker. You can conveniently start a Kafka Docker container by executing the `start_kafka_broker_locally.sh` shell script:

```sh
./scripts/start_kafka_broker_locally.sh
```

```console
[+] Running 2/2
 ⠿ Network scripts_default  Created                                                                                                             0.1s
 ⠿ Container bitnami_kafka  Started 
```

#### Start the application

To start the application, execute the following command:
```sh
faststream run  app.application:app
```
```console
2023-09-15 13:41:21,948 INFO     - FastStream app starting...
2023-09-15 13:41:22,144 INFO     -      |            - Starting publishing:
2023-09-15 13:41:22,144 INFO     - FastStream app started successfully! To exit press CTRL+C
Topic new_crypto_pricenew_data not found in cluster metadata
```

### Calculate the moving average app

This application will calculate the mean price of the last three messages received from the 'new_data' topic for each cryptocurrency. Afterwards, the calculated mean price will be published to the 'price_mean' topic.


Here is the full description of the desired application:

In [6]:
# | hide

with open('../../docs_src/tutorial/calculate_mean_description.txt', 'r') as file:
    description = file.read()

description = f"""
```text
{description} 
```
"""

In [7]:
# | echo: false

md(description)


```text
Create a FastStream application for consuming messages from the new_data topic. 
This topic needs to use a partition key.

new_data messages use JSON with two attributes 
(create class CryptoPrice with these attributes):
- price: non-negative float (it represents the current price of the crypto)
- crypto_currency: string (it represents the cryptocurrency e.g. BTC, ETH...)

The application should save each message to a dictionary (global variable) 
- partition key should be used as a dictionary key 
  and value should be a List of prices.
  
Keep only the last 100 messages in the dictionary.

If there are fewer than 3 messages for a given partition key,
do not publish any messages.

Otherwise, Calculate the price mean of the last 3 messages
for the given partition key.

Publish the price mean to the price_mean topic and use 
the same partition key that the new_data topic is using.
 
```


To create a `faststream` application inside the `calculate_mean_app` directory, first, copy the previous description and paste it into the `description_calculate_mean.txt` file.

Next, run the following command:

```sh
faststream_gen -i description_calculate_mean.txt -o calculate_mean_app
```
```console
✨  Generating a new FastStream application!
 ✔ Application description validated. 
 ✔ FastStream app skeleton code generated. 
 ✔ The app and the tests are generated. 
 ✔ New FastStream project created. 
 ✔ Integration tests were successfully completed. 
 Tokens used: 13367
 Total Cost (USD): $0.04147
✨  All files were successfully generated!
```

This command will generate `calculate_mean_app` directory with `app/application.py` and `tests/test_application.py` inside.

`app/application.py`:

In [8]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/calculate_mean_app.py', language='python')
md(f"```python\n{code}\n```")

```python
from typing import Dict, List

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import Context, ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(
        ..., examples=[50000], description="Current price of the cryptocurrency"
    )
    crypto_currency: str = Field(
        ..., examples=["BTC"], description="Cryptocurrency symbol"
    )


publisher = broker.publisher("price_mean")


@app.on_startup
async def app_setup(context: ContextRepo):
    message_history: Dict[str, List[float]] = {}
    context.set_global("message_history", message_history)


@broker.subscriber("new_data")
async def on_new_data(
    msg: CryptoPrice,
    logger: Logger,
    message_history: Dict[str, List[float]] = Context(),
    key: bytes = Context("message.raw_message.key"),
) -> None:
    logger.info(f"New data received: {msg=}")

    partition_key = key.decode("utf-8")
    if partition_key not in message_history:
        message_history[partition_key] = []

    message_history[partition_key].append(msg.price)

    if len(message_history[partition_key]) > 100:
        message_history[partition_key].pop(0)

    if len(message_history[partition_key]) >= 3:
        price_mean = sum(message_history[partition_key][-3:]) / 3
        await publisher.publish(price_mean, key=key)

```

`tests/test_application.py`

In [9]:
# | echo: false

code = Code(filename='../../docs_src/tutorial/calculate_mean_test.py', language='python')
md(f"```python\n{code}\n```")

```python
import pytest

from faststream import Context, TestApp
from faststream.kafka import TestKafkaBroker

from app.application import CryptoPrice, app, broker


@broker.subscriber("price_mean")
async def on_price_mean(
    msg: float, key: bytes = Context("message.raw_message.key")
):
    pass


@pytest.mark.asyncio
async def test_price_mean_calculation():
    async with TestKafkaBroker(broker):
        async with TestApp(app):
            await broker.publish(
                CryptoPrice(price=100, crypto_currency="BTC"),
                "new_data",
                key=b"partition_key",
            )
            await broker.publish(
                CryptoPrice(price=200, crypto_currency="BTC"),
                "new_data",
                key=b"partition_key",
            )
            await broker.publish(
                CryptoPrice(price=300, crypto_currency="BTC"),
                "new_data",
                key=b"partition_key",
            )
            await broker.publish(
                CryptoPrice(price=400, crypto_currency="BTC"),
                "new_data",
                key=b"partition_key",
            )
            await broker.publish(
                CryptoPrice(price=500, crypto_currency="BTC"),
                "new_data",
                key=b"partition_key",
            )

            on_price_mean.mock.assert_called_with(400.0)
```

### Retrieve and publish app

### Calculate the moving average app

Open **new terminal**, navigate to the "faststream_gen_tutorial" directory and make sure that you have activated virtual environment.

To start the `calculate_mean_app`, run the following command:
```sh
faststream run calculate_mean_app.app.application:app
```
```console
2023-09-15 13:56:47,245 INFO     - FastStream app starting...
2023-09-15 13:56:47,428 INFO     - new_data |            - `OnNewData` waiting for messages
2023-09-15 13:56:47,621 INFO     - FastStream app started successfully! To exit press CTRL+C
2023-09-15 13:56:48,314 INFO     - new_data | 13675-1694 - Received
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - msg=NewData(price=1624.235, currency='ETH')
2023-09-15 13:56:48,315 INFO     - new_data | 13675-1694 - Processed
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Received
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - msg=NewData(price=26485.545, currency='BTC')
2023-09-15 13:56:48,316 INFO     - new_data | 13676-1694 - Processed
2023-09-15 13:56:50,491 INFO     - new_data | 13677-1694 - Received
2023-09-15 13:56:50,492 INFO     - new_data | 13677-1694 - msg=NewData(price=1624.235, currency='ETH')
...
```

You can see in the terminal the that the application is reading the messages from the `price_mean` topic.

### Subscribe directly to local kafka broker topic

Open **new terminal**, navigate to the "faststream_gen_tutorial" directory and make sure that you have activated virtual environment.

To check if the `calculate_mean_app` is publishing messages to the `price_mean` topic, run the following comand:

 ```sh
./retrieve_publish_app/scripts/subscribe_to_kafka_broker_locally.sh price_mean
```
```console
BTC     26405.745
ETH     1621.3733333333332
BTC     26404.865
ETH     1621.375
BTC     26404.865
...
```

To stop the Kafka broker after analyzing the mean price of cryptocurrencies, you can execute the following commands:
```sh
cd retrieve_publish_app 
./scripts/stop_kafka_broker_locally.sh
```
```console
[+] Running 2/2
 ⠿ Container bitnami_kafka  Removed                                                                            1.2s
 ⠿ Network scripts_default  Removed 
```

## Next steps

Congratulations! You have successfully completed this tutorial and gained a new set of skills. Now that you have learned how to use `faststream-gen`, try it out with your own example!